In [ ]:
1 - définition du fonction qui évalue le cout métier global de toutes mes prédictions

2 - tester différents seuils, et choisi celui qui minimise la fonction cout

3 - tester le modèle final avec ce seuil avec AUC-ROC, accuracy, rappel, F1

In [ ]:
# Define a work cost function
def cost_metric(y_true, y_pred, cost_fn=cost_fn, cost_fp=cost_fp):
    _, fp, fn, _ = confusion_matrix(y_true, y_pred).ravel()
    cost = (fn * cost_fn) + (fp * cost_fp)
    return cost

## Il faut la normaliser entre 2 valeurs pour que ça soit comparable entre 2 jeux de données différents

In [ ]:
def optimize_threshold(y_true, y_proba, cost_fn=cost_fn, cost_fp=cost_fp):
    _, _, thresholds = roc_curve(y_true, y_proba)
    costs = []
    for threshold in thresholds:
        y_pred = (y_proba <= threshold).astype(int)
        cost = cost_metric(y_true, y_pred, cost_fn, cost_fp)
        costs.append(cost)
    optimal_idx = np.argmin(costs)
    optimal_threshold = thresholds[optimal_idx]
    return optimal_threshold

In [ ]:
model = pickle.load('model')

# LightGBM classifier
custom_model = CustomThresholdModel(model, threshold)

# Prédire les probabilités sur l'ensemble de test
scores, proba = custom_model.predict(X_test)[:, 1]

# Calculer l'AUC-ROC sur l'ensemble de test
test_auc_roc = roc_auc_score(y_test, y_pred_proba)
print(f"Test AUC-ROC: {test_auc_roc}")